# Add mechanical-coupled diffusion to PyBaMM
### 1. Theoratical background:

Here the mechanical-coupled diffusion is incorporated into a pseudo-2D (P2D) model for lithium-ion betteries. We will start from the model for a single electrode particle in Ref. [1] to full battery model in Ref. [2]. Only the key equations are introduced, while the details can be found in Refs. [1,2].

For a spherical electrpde particle, the radial stress $\sigma_\text{r}$, tangential stress $\sigma_\text{t}$ and displacement $u$ are determined by lithium concentration profiles in Ref. [1] as
$$
\sigma_\text{r}=\frac{2\Omega E}{(1-\nu)}[c_\text{avg}(R_i)-c_\text{avg}(r)]\\
\sigma_\text{t}=\frac{\Omega E}{(1-\nu)}[2c_\text{avg}(R_i)+c_\text{avg}(r)-\bar{c}/3)\\
u=\frac{(1+\nu)}{(1-\nu)}\Omega rc_\text{avg}(r)+\frac{2(1-2\nu)}{(1-\nu)}\Omega rc_\text{avg}(R_i)\\
\label{eqn:stress-particle}\tag{1}
$$
where $\Omega$ is the partial molar volume, $E$ is the Young's modulus, $\nu$ is the Possion's ratio, $R_i$ is the radius of the particle and $c_\text{avg}(r)$ is the average concentration for a fraction (with a radius $r$) of the particle with $c_\text{avg}(r)=\frac{1}{3r^3}\int^r_0 \bar{c}r^2\text{d}r$. In Ref. [1], the electrochemical potential is modified by including the mechanical strain energy, then the diffusion equation for species becomes 
$$
\frac{\text{d}c}{\text{d}t}+\nabla \cdot \textbf{J}=0, \text{where } \textbf{J}=-D(\nabla c-\frac{\Omega \bar{c}}{RT}\nabla \sigma_h)\label{eqn:solute-diffusion}\tag{2}\\
$$
$\sigma_h$ is the hydrostatic stress $\sigma_h=(\sigma_\text{r}+2\sigma_t)/3$, and $\bar{c}$ is the change of lithium concentration from the reference value $c_\text{ref}$, at which there is no stress, with $\bar{c}=c-c_\text{ref}$. Considering the spherial sysmetry and the solution of stresses in Eq. (\ref{eqn:stress-particle}), the lithium flux in 1D is rewritten as 
$$
\textbf{J}=-D(1+\theta \bar{c})\frac{\partial c}{\partial r}\tag{3}\\
$$
where $\theta=\frac{\Omega}{RT}\frac{2\Omega E}{9(1-\nu)}$ with unit of [m^3/mol]. In the above equation, the diffusion equation in the Newman's model without considering mechanical effects is achieved by seting $\theta=0$.

From a P2D model, lithium concentration profiles can be obatained over electrodes and inside electrode particles. Then stress distribution can be calculated by applying Eq. (\ref{eqn:stress-particle}) at the specific locations [2]. The thickness change of battery cells can be predicted using the mechanical model above. The volume change of a electrode particle by lithium (de)intercalation is estimated by
$$
\frac{\text{d}V_i}{V_i}\approx \frac{3\text{d}R_i}{R_i}=\frac{3 u(R_i)}{R_i}=\Omega c_{i,\text{avg}}\tag{4}\\
$$
Becasue the thickness of electrode layers is much smaller than the length and width of the cell, the thickness change is estimated by the volume change of the cell $\frac{\text{d}L}{L}\approx\frac{\epsilon_\text{s}\text{d}V}{V}$, where $\epsilon_\text{s}$ is the volume fraction of active materials. The thickness change of the cell by lithium (de)intercalation $\text{d}L_\text{int}$ is 
$$
\text{d}L_\text{int}=n \cdot \sum_{i=1}^{n_\text{p}+n_\text{n}} \epsilon_\text{s}\Omega c_{i,\text{avg}L_i}\tag{5}\\
$$
where $n_\text{p}$ and $n_\text{n}$ are the number of elements in cathode and anode respectively, $n$ is the number of cathode and anode layers in the pouch cell, $L_i$ is the fraction of the layer respented by the particle. A constant partial molar volume $\Omega$ results in a linear relationship between the thickness change and state of charge, while a non-linear relationship can be obtained by considering a concentration dependant $\Omega$ in Ref. [2].  


### Reference:

[1] Zhang, X., Shyy, W. and Sastry, A.M., 2007. Numerical simulation of intercalation-induced stress in Li-ion battery electrode particles. Journal of the Electrochemical Society, 154(10), pp.A910-A916. <a href="doi: 10.1149/1.2759840">doi: 10.1149/1.2759840</a>

[2] Ai, W., Kraft, L., Sturm, J., Jossen, A., & Wu, B. (2020). Electrochemical Thermal-Mechanical Modelling of Stress Inhomogeneity in Lithium-Ion Pouch Cells. Journal of The Electrochemical Society, 167(1), 013512. <a href="doi: 10.1149/2.0122001JES">doi: 10.1149/2.0122001JES</a>

### 2. Implementatin and calling the model in PyBaMM

first import PyBaMM and change our working directory to the root of the pybamm folder

In [1]:
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

In [2]:
# load model
model = pybamm.lithium_ion.DFN()

# How to change the choice of submodels?

# create geometry
geometry = model.default_geometry

# load parameter values and process model and geometry
param = model.default_parameter_values
param.process_model(model)
param.process_geometry(geometry)

In [8]:
# set mesh
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise model
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

In [9]:
# solve model
solver = model.default_solver
t_eval = np.linspace(0, 1, 300)
solution = solver.solve(model, t_eval)


In [10]:
quick_plot = pybamm.QuickPlot(model, mesh, solution)

import ipywidgets as widgets
widgets.interact(quick_plot.plot, t=widgets.FloatSlider(min=0,max=solution.t[-1],step=0.05,value=0));

interactive(children=(FloatSlider(value=0.0, description='t', max=0.1605351170568562, step=0.05), Output()), _…